# Ship or Iceberg
https://www.kaggle.com/c/statoil-iceberg-classifier-challenge

In [123]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [148]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

import numpy as np
import pandas as pd
import sklearn.preprocessing

import matplotlib.pyplot as plt

# Data

Load and preprocess data

In [126]:
datadir = "/home/christopher/Data/data/ml/ship-or-iceberg/"

with open(datadir + "train.json") as f:
    train = pd.read_json(f).set_index("id")
train.head()

,band_1,band_2,inc_angle,is_iceberg
id,,,,
dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0


In [143]:
band1 = np.array([np.array([i]) for i in train["band_1"]], dtype=np.float32)
band1 = band1.reshape(len(band1), 1, 75, 75)
band2 = np.array([np.array([i]) for i in train["band_2"]], dtype=np.float32)
band2 = band2.reshape(len(band2), 1, 75, 75)

train_data = np.concatenate((band1, band2), axis=1)
labels = np.zeros((len(train), 2), dtype=np.float32)
for i, is_iceberg in enumerate(train["is_iceberg"]):
    labels[i][is_iceberg] = 1

In [144]:
td = train_data.reshape(len(train_data), -1)
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(td)
train_data = scaler.transform(td)
train_data = train_data.reshape(len(train_data), 2, 75, 75)

# Data summary

In [161]:
train_data = autograd.Variable(torch.from_numpy(train_data))
labels = autograd.Variable(torch.from_numpy(labels))

RuntimeError: from_numpy expects an np.ndarray but got Variable

In [162]:
train_data.shape

torch.Size([1604, 2, 75, 75])

# Neural Network

In [187]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(2, 3, 4) # in_chan, out_chan, kernel_size
        self.pool = nn.MaxPool2d(3, 3)
        self.lin1 = nn.Linear(3 * 24 * 24, 100) # in_feat, out_feat
        self.lin2 = nn.Linear(100, 2)
    
    def forward(self, x):
        # 2x75x75 -> 3x72x72 -> 3x24x24 -> 3*24*24 -> 100 -> 2
        x = self.conv1(x)
        x = self.pool(x)
        x = x.view(-1, 3*24*24)
        x = self.lin1(x) 
        x = self.lin2(x)
        return x
net = Net() # Why does moving net into the next block mess everything up?
loss_fn = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
net.cuda()
for epoch in range(10):
    batch_size = 10
    running_loss = 0.0
    for start in np.arange(0, len(train_data) - batch_size, batch_size):
        batch_input, batch_labels = train_data[start:start+batch_size].cuda(), labels[start:start+batch_size].cuda()
        optimizer.zero_grad()
    
        batch_output = net(batch_input)
        batch_loss = loss_fn(batch_output, batch_labels)
        batch_loss.backward()
        optimizer.step()
        running_loss += batch_loss.data[0]
        
    print(running_loss)

In [172]:
print(net(train_data[1600:]))
print(labels[1600:])

Variable containing:
 1.4921 -0.7884
 0.6099  0.4212
 0.5932  0.4677
 0.6433  0.4454
[torch.FloatTensor of size 4x2]

Variable containing:
 1  0
 1  0
 1  0
 1  0
[torch.FloatTensor of size 4x2]

